In [1]:
# ========== N-BEATS Standardized Version ==========
# Train with standardized data, automatically inverse-transform y after prediction
# Requires corresponding scaler_y_window_*.pkl file for inverse transformation

# ========== Basic Libraries ==========
import numpy as np
import pandas as pd
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import joblib
import warnings
warnings.filterwarnings('ignore')

# ========== Evaluation Metrics ==========
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import TimeSeriesSplit

# ========== Hyperparameter Tuning ==========
import optuna
optuna.logging.set_verbosity(optuna.logging.WARNING)

# ========== Visualization ==========
import matplotlib.pyplot as plt

# ========== Global Configuration ==========
np.random.seed(42)
torch.manual_seed(42)

# ========== MPS Acceleration Configuration ==========
if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("Using MPS (Metal Performance Shaders)")
elif torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using CUDA")
else:
    device = torch.device("cpu")
    print("Using CPU")

torch.backends.cudnn.benchmark = True
torch.set_float32_matmul_precision('medium')


Using MPS (Metal Performance Shaders)


In [2]:
# ========== Data Loading ==========
def load_datasets(npz_path="/Users/june/Documents/University of Manchester/Data Science/ERP/Project code/1_Data_Preprocessing/all_window_datasets_scaled.npz"):
    data = np.load(npz_path, allow_pickle=True) 
    datasets = {}
    for key in data.files:
        datasets[key] = data[key]
    return datasets

def load_y_scaler(window_size, scaler_dir="/Users/june/Documents/University of Manchester/Data Science/ERP/Project code/1_Data_Preprocessing/"):
    """Load the y scaler for the corresponding window size"""
    scaler_path = os.path.join(scaler_dir, f"scaler_y_window_{window_size}.pkl")
    if os.path.exists(scaler_path):
        scaler = joblib.load(scaler_path)
        print(f"Loaded y scaler for window {window_size}")
        return scaler
    else:
        print(f"Warning: Y scaler not found for window {window_size}: {scaler_path}")
        return None

def prepare_sequences(X, y, lookback):
    """Prepare sequence data for N-BEATS training"""
    if len(X) < lookback:
        return np.array([]), np.array([])
    
    X_seq, y_seq = [], []
    
    for i in range(lookback, len(X)):
        X_seq.append(X[i])
        if y is not None:
            y_seq.append(y[i])
    
    if y is None:
        return np.array(X_seq), np.array([])
    return np.array(X_seq), np.array(y_seq)


In [ ]:
# ========== Evaluation Metrics ==========
def r2_zero(y_true, y_pred):
    """
    Calculate zero-based R² (baseline is 0)
    y_true: true values array (N,)
    y_pred: predicted values array (N,)
    """
    rss = np.sum((y_true - y_pred)**2)  
    tss = np.sum(y_true**2)            
    return 1 - rss / tss

def calc_directional_metrics(y_true, y_pred, permnos=None):
    """Calculate directional metrics"""
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)

    if permnos is None:
        s_true = np.sign(y_true)
        s_pred = np.sign(y_pred)
        mask = s_true != 0
        s_true = s_true[mask]
        s_pred = s_pred[mask]

        overall_acc = np.mean(s_true == s_pred)
        up_mask = s_true > 0
        down_mask = s_true < 0
        up_acc = np.mean(s_true[up_mask] == s_pred[up_mask]) if np.any(up_mask) else 0
        down_acc = np.mean(s_true[down_mask] == s_pred[down_mask]) if np.any(down_mask) else 0
    else:
        df = pd.DataFrame({"permno": permnos, "yt": y_true, "yp": y_pred})
        overall_accs = []
        up_accs = []
        down_accs = []

        for _, g in df.groupby("permno"):
            s_true = np.sign(g["yt"].values)
            s_pred = np.sign(g["yp"].values)
            mask = s_true != 0
            s_true = s_true[mask]
            s_pred = s_pred[mask]
            if len(s_true) == 0:
                continue
            overall_accs.append(np.mean(s_true == s_pred))

            up_mask = s_true > 0
            down_mask = s_true < 0
            up_accs.append(np.mean(s_true[up_mask] == s_pred[up_mask]) if np.any(up_mask) else np.nan)
            down_accs.append(np.mean(s_true[down_mask] == s_pred[down_mask]) if np.any(down_mask) else np.nan)

        overall_acc = np.nanmean(overall_accs)
        up_acc = np.nanmean(up_accs)
        down_acc = np.nanmean(down_accs)

    return overall_acc, up_acc, down_acc

def regression_metrics(y_true, y_pred, k, meta=None, permnos=None):
    """Evaluation metrics for deep learning models"""
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    n = len(y_true)

    r2 = r2_zero(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)

    dir_acc, up_acc, down_acc = calc_directional_metrics(y_true, y_pred, permnos)

    metrics = {
        "R2_zero": r2,
        "RMSE": rmse,
        "MAE": mae,
        "MSE": mse,
        "Directional Accuracy": dir_acc,
        "Up_Directional_Acc": up_acc,
        "Down_Directional_Acc": down_acc
    }

    # Market cap group analysis
    if meta is not None and "MKTCAP_PERCENTILE" in meta:
        top_mask = meta["MKTCAP_PERCENTILE"] >= 0.75
        bottom_mask = meta["MKTCAP_PERCENTILE"] <= 0.25

        if np.any(top_mask):
            yt_top = y_true[top_mask]
            yp_top = y_pred[top_mask]
            perm_top = permnos[top_mask] if permnos is not None else None
            r2_top = r2_zero(yt_top, yp_top)
            rmse_top = np.sqrt(mean_squared_error(yt_top, yp_top))
            mae_top = mean_absolute_error(yt_top, yp_top)
            mse_top = mean_squared_error(yt_top, yp_top)
            dir_top, up_top, down_top = calc_directional_metrics(yt_top, yp_top, perm_top)
            metrics.update({
                "Top25_R2_zero": r2_top,
                "Top25_RMSE": rmse_top,
                "Top25_MAE": mae_top,
                "Top25_MSE": mse_top,
                "Top25_Dir_Acc": dir_top,
                "Top25_Up_Acc": up_top,
                "Top25_Down_Acc": down_top
            })

        if np.any(bottom_mask):
            yt_bot = y_true[bottom_mask]
            yp_bot = y_pred[bottom_mask]
            perm_bot = permnos[bottom_mask] if permnos is not None else None
            r2_bot = r2_zero(yt_bot, yp_bot)
            rmse_bot = np.sqrt(mean_squared_error(yt_bot, yp_bot))
            mae_bot = mean_absolute_error(yt_bot, yp_bot)
            mse_bot = mean_squared_error(yt_bot, yp_bot)
            dir_bot, up_bot, down_bot = calc_directional_metrics(yt_bot, yp_bot, perm_bot)
            metrics.update({
                "Bottom25_R2_zero": r2_bot,
                "Bottom25_RMSE": rmse_bot,
                "Bottom25_MAE": mae_bot,
                "Bottom25_MSE": mse_bot,
                "Bottom25_Dir_Acc": dir_bot,
                "Bottom25_Up_Acc": up_bot,
                "Bottom25_Down_Acc": down_bot
            })

    return metrics


In [ ]:
# ========== Save model and metrics ==========
def save_model(model, name, window, path="models/"):
    os.makedirs(path, exist_ok=True)
    torch.save(model.state_dict(), os.path.join(path, f"{name}_w{window}.pth"))

def save_metrics(metrics_dict, name, window, path="results.csv"):
    row = pd.DataFrame([metrics_dict])
    row.insert(0, "Model", name)
    row.insert(1, "Window", window)

    if os.path.exists(path):
        df = pd.read_csv(path)
        df = df[~((df["Model"] == name) & (df["Window"] == window))]
        df = pd.concat([df, row], ignore_index=True)
        df.to_csv(path, index=False)
    else:
        row.to_csv(path, index=False)

def save_predictions(model_name, window_size, y_true, y_pred, permnos, path="predictions/"):
    os.makedirs(path, exist_ok=True)
    
    df = pd.DataFrame({
        "PERMNO": permnos,
        "y_true": y_true,
        "y_pred": y_pred
    })

    filename = f"{model_name}_w{window_size}.csv"
    df.to_csv(os.path.join(path, filename), index=False)


In [ ]:
# N-BEATS model architecture
class NBeatsBlock(nn.Module):
    def __init__(self, input_size, theta_size, basis_size, layers, layer_size):
        super().__init__()
        self.layers = nn.ModuleList([nn.Linear(input_size, layer_size)] + 
                                   [nn.Linear(layer_size, layer_size) for _ in range(layers-1)])
        self.basis_parameters = nn.Linear(layer_size, theta_size)
        self.input_size = input_size
        self.theta_size = theta_size
        self.basis_size = basis_size
        
    def forward(self, x):
        for layer in self.layers:
            x = torch.relu(layer(x))
        theta = self.basis_parameters(x)
        backcast = theta[:, :self.input_size]
        forecast = theta[:, self.input_size:self.input_size+1]
        return backcast, forecast

class NBeatsNet(nn.Module):
    def __init__(self, input_size, stacks=2, blocks_per_stack=2, layers=4, layer_size=128):
        super().__init__()
        self.input_size = input_size
        self.stacks = nn.ModuleList()
        for _ in range(stacks):
            stack = nn.ModuleList()
            for _ in range(blocks_per_stack):
                stack.append(NBeatsBlock(input_size, input_size + 1, input_size, layers, layer_size))
            self.stacks.append(stack)
    
    def forward(self, x):
        residual = x
        forecast = 0
        for stack in self.stacks:
            for block in stack:
                backcast, block_forecast = block(residual)
                residual = residual - backcast
                forecast = forecast + block_forecast
        return forecast

def train_step(model, criterion, optimizer, X_batch, y_batch):
    model.train()
    optimizer.zero_grad()
    predictions = model(X_batch).squeeze()
    loss = criterion(predictions, y_batch)
    loss.backward()
    optimizer.step()
    return loss.item()

if device.type == "cuda":
    train_step = torch.compile(train_step)


In [ ]:
# ========== Hyperparameter Tuning ==========
TUNED_MODELS = {"NBEATS"}

def tune_nbeats_with_optuna(X, y, window, n_trials=10):
    """N-BEATS hyperparameter tuning using TimeSeriesSplit"""
    if len(y) < 100:
        return {
            'stacks': 2, 
            'blocks_per_stack': 2, 
            'layers': 2, 
            'layer_size': 128, 
            'learning_rate': 0.001, 
            'batch_size': 32,
            'max_epochs': 25,
            'warm_start_epochs': 15
        }
    
    print(f"    [Hyperparameter Tuning] Running Optuna optimization for window={window}")
    print(f"    [Device Setting] Using CPU for hyperparameter tuning (faster for small windows)")
    
    tuning_device = torch.device("cpu")
    
    tscv = TimeSeriesSplit(n_splits=3)
    features_per_timestep = X.shape[1] // window
    
    def objective(trial):
        try:
            stacks = trial.suggest_int("stacks", 1, 3)
            blocks_per_stack = trial.suggest_int("blocks_per_stack", 1, 3)
            layers = 2
            layer_size = 128
            lr = trial.suggest_float("lr", 1e-5, 5e-4, log=True)
            batch_size = trial.suggest_categorical("batch_size", [32, 64])
            
            cv_scores = []
            for train_idx, val_idx in tscv.split(X):
                X_tr, X_val = X[train_idx], X[val_idx]
                y_tr, y_val = y[train_idx], y[val_idx]
                
                if len(X_tr) < 50 or len(X_val) < 10:
                    continue
                
                lookback = min(20, len(X_tr)//4)
                X_seq_tr, y_seq_tr = prepare_sequences(X_tr, y_tr, lookback)
                X_seq_val, y_seq_val = prepare_sequences(X_val, y_val, lookback)
                
                if len(X_seq_tr) == 0 or len(X_seq_val) == 0:
                    continue
                
                X_tensor_tr = torch.FloatTensor(X_seq_tr).to(tuning_device)
                y_tensor_tr = torch.FloatTensor(y_seq_tr).to(tuning_device)
                X_tensor_val = torch.FloatTensor(X_seq_val).to(tuning_device)
                y_tensor_val = torch.FloatTensor(y_seq_val).to(tuning_device)
                
                input_size = X_seq_tr.shape[-1]
                model = NBeatsNet(input_size, stacks, blocks_per_stack, layers, layer_size).to(tuning_device)
                criterion = nn.MSELoss()
                optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-5)
                
                dataset = TensorDataset(X_tensor_tr, y_tensor_tr)
                dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, pin_memory=False, num_workers=0)
                
                model.train()
                for epoch in range(5):
                    for X_batch, y_batch in dataloader:
                        train_step(model, criterion, optimizer, X_batch, y_batch)
                
                model.eval()
                with torch.no_grad():
                    val_pred = model(X_tensor_val).squeeze().cpu().numpy()
                    mse = mean_squared_error(y_seq_val, val_pred)
                    cv_scores.append(mse)
                
                del model, optimizer, criterion, X_tensor_tr, y_tensor_tr, X_tensor_val, y_tensor_val
                if tuning_device.type == 'cuda':
                    torch.cuda.empty_cache()
                elif tuning_device.type == 'mps':
                    torch.mps.empty_cache()
            
            return np.mean(cv_scores) if cv_scores else float('inf')
        except Exception as e:
            return float('inf')
    
    study = optuna.create_study(direction="minimize", sampler=optuna.samplers.TPESampler(seed=42))
    study.optimize(objective, n_trials=n_trials)
    
    print(f"    [Tuning Completed] Switching back to {device} for model training")
    
    if study.best_trial is None:
        return {
            'stacks': 2, 
            'blocks_per_stack': 2, 
            'layers':2, 
            'layer_size': 128, 
            'learning_rate': 0.001, 
            'batch_size': 32,
            'max_epochs': 25,
            'warm_start_epochs': 15
        }
    
    best_params = study.best_params.copy()
    best_params['learning_rate'] = best_params.pop('lr')
    best_params['max_epochs'] = 25
    best_params['warm_start_epochs'] = 15
    best_params['layers']             = 2                         
    best_params['layer_size']         = 128  
    
    print(f"    [Optuna] NBEATS best_params={best_params}")
    return best_params


In [ ]:
# ========== Model Training and Prediction ==========
def train_nbeats_model(X_train, y_train, X_test, y_test, best_params, max_epochs=50, y_scaler=None):
    """Train N-BEATS model and return aligned test data, support inverse scaling for y"""
    try:
        lookback = min(20, len(X_train)//4)
        X_seq_train, y_seq_train = prepare_sequences(X_train, y_train, lookback)
        X_seq_test, y_seq_test = prepare_sequences(X_test, y_test, lookback)
        
        if len(X_seq_train) == 0:
            return None, None, None
        
        X_tensor_train = torch.FloatTensor(X_seq_train).to(device)
        y_tensor_train = torch.FloatTensor(y_seq_train).to(device)
        X_tensor_test = torch.FloatTensor(X_seq_test).to(device)
        
        input_size = X_seq_train.shape[-1]
        model = NBeatsNet(
            input_size=input_size,
            stacks=best_params['stacks'],
            blocks_per_stack=best_params['blocks_per_stack'],
            layers=best_params['layers'],
            layer_size=best_params['layer_size']
        ).to(device)
        
        criterion = nn.MSELoss()
        lr_key = 'learning_rate' if 'learning_rate' in best_params else 'lr'
        optimizer = optim.AdamW(model.parameters(), lr=best_params[lr_key], weight_decay=1e-5)
        
        dataset = TensorDataset(X_tensor_train, y_tensor_train)
        pin_memory = device.type == "cuda"
        dataloader = DataLoader(
            dataset, 
            batch_size=best_params['batch_size'], 
            shuffle=True, 
            pin_memory=pin_memory,
            num_workers=0
        )
        
        model.train()
        for epoch in range(max_epochs):
            epoch_loss = 0
            for X_batch, y_batch in dataloader:
                loss = train_step(model, criterion, optimizer, X_batch, y_batch)
                epoch_loss += loss
            
            if epoch > 10 and epoch_loss < 1e-6:
                break
        
        model.eval()
        with torch.no_grad():
            if len(X_seq_test) > 0:
                y_pred_tensor = model(X_tensor_test).squeeze()
                y_pred = y_pred_tensor.cpu().numpy()
                
                if len(y_pred) < len(y_test):
                    y_test_aligned = y_test[-len(y_pred):]
                else:
                    y_pred = y_pred[:len(y_test)]
                    y_test_aligned = y_test
                
                if y_scaler is not None:
                    y_pred_reshaped = y_pred.reshape(-1, 1)
                    y_pred_inverse = y_scaler.inverse_transform(y_pred_reshaped).flatten()
                    
                    y_test_reshaped = y_test_aligned.reshape(-1, 1)
                    y_test_inverse = y_scaler.inverse_transform(y_test_reshaped).flatten()
                    
                    print(f"Applied inverse scaling: pred range [{y_pred_inverse.min():.6f}, {y_pred_inverse.max():.6f}], test range [{y_test_inverse.min():.6f}, {y_test_inverse.max():.6f}]")
                    return model, y_pred_inverse, y_test_inverse
                else:
                    print("No y_scaler provided, using standardized values")
                    return model, y_pred, y_test_aligned
            else:
                return model, np.array([]), np.array([])
        
    except Exception as e:
        print(f"Training failed: {e}")
        return None, None, None

def get_model(name: str, best_params=None):
    """Get default model parameters"""
    if name == "NBEATS":
        if best_params:
            return best_params
        else:
            return {
                'stacks': 2, 
                'blocks_per_stack': 2, 
                'layers': 2, 
                'layer_size': 128, 
                'learning_rate': 0.001, 
                'batch_size': 32,
                'max_epochs': 25,
                'warm_start_epochs': 15
            }
    
    raise ValueError(f"Unexpected model: {name}")


In [ ]:
# ========== Main training and evaluation logic ==========
def train_and_evaluate(model_name, window_size,
                       X_train, y_train, X_test, y_test,
                       permnos_train, permnos_test, meta=None, shared_params=None):
    """Train and evaluate N-BEATS model"""
    
    y_scaler = load_y_scaler(window_size)
    
    if model_name in TUNED_MODELS:
        if shared_params is None:
            print(f"[Hyperparameter Tuning] Running Optuna optimization for window={window_size}")
            best_params = tune_nbeats_with_optuna(X_train, y_train, window_size, n_trials=10)
            print(f"[Optuna] {model_name} best_params={best_params}")
            model_params = get_model(model_name, best_params)
        else:
            print(f"[Shared Parameters] Using optimized params from window=5 for window={window_size}")
            model_params = get_model(model_name, shared_params)
    else:
        model_params = get_model(model_name)

    fitted_model, y_pred, y_test_aligned = train_nbeats_model(X_train, y_train, X_test, y_test, model_params, y_scaler=y_scaler)
    
    if fitted_model is None or y_pred is None or y_test_aligned is None:
        print(f"[Skip Model] {model_name} failed to fit. Skipping.")
        return None

    if len(y_pred) == 0:
        print(f"[Skip Model] {model_name} no valid predictions. Skipping.")
        return None

    if len(y_test_aligned) < len(y_test):
        offset = len(y_test) - len(y_test_aligned)
        meta_aligned = meta.iloc[offset:] if meta is not None else None
        permnos_test_aligned = permnos_test[offset:] if permnos_test is not None else None
    else:
        meta_aligned = meta
        permnos_test_aligned = permnos_test

    k = X_test.shape[1]
    metrics = regression_metrics(y_test_aligned, y_pred, k, meta=meta_aligned, permnos=permnos_test_aligned)

    save_model(fitted_model, model_name, window_size)
    save_metrics(metrics, model_name, window_size)
    save_predictions(model_name, window_size, y_test_aligned, y_pred, permnos_test_aligned)

    print(f"Completed {model_name} w={window_size}: MSE={metrics['MSE']:.6f}, Dir_Acc={metrics['Directional Accuracy']:.4f}")
    
    if device.type == 'mps':
        torch.mps.empty_cache()
    elif device.type == 'cuda':
        torch.cuda.empty_cache()
    
    if shared_params is None and model_name in TUNED_MODELS:
        return metrics, best_params
    else:
        return metrics, None


In [9]:
# Main dispatcher function
def loop_all_models():
    """
    Loop through all N-BEATS models and window sizes.
    Use standardized data for training, automatically inverse-transform y.
    Hyperparameter tuning is only performed on window=5, and the parameters are shared with other windows.
    """
    datasets = load_datasets()
    model_list = ["NBEATS"]
    window_sizes = [5, 21, 252, 512]
    
    shared_hyperparams = None

    for window in window_sizes:
        X_train = datasets[f"X_train_{window}"]
        y_train = datasets[f"y_train_{window}"]
        X_test = datasets[f"X_test_{window}"]
        y_test = datasets[f"y_test_{window}"]

        meta_train_dict = datasets[f"meta_train_{window}"].item()
        meta_test_dict = datasets[f"meta_test_{window}"].item()

        meta_train = pd.DataFrame.from_dict(meta_train_dict)
        meta_test = pd.DataFrame.from_dict(meta_test_dict)

        permnos_train = meta_train["PERMNO"].values
        permnos_test = meta_test["PERMNO"].values

        for model_name in model_list:
            print(f"Training {model_name} on Window = {window}")
            
            if window == 5:
                result = train_and_evaluate(
                    model_name, window,
                    X_train, y_train, X_test, y_test,
                    permnos_train, permnos_test,  
                    meta_test, shared_params=None
                )
                if result is not None:
                    metrics, optimized_params = result
                    shared_hyperparams = optimized_params
                    print(f"[Shared Hyperparams] Optimized on window=5: {shared_hyperparams}")
                else:
                    print(f"[Error] Failed to train {model_name} on window={window}")
                    continue
            else:
                result = train_and_evaluate(
                    model_name, window,
                    X_train, y_train, X_test, y_test,
                    permnos_train, permnos_test,  
                    meta_test, shared_params=shared_hyperparams
                )
                if result is None:
                    print(f"[Error] Failed to train {model_name} on window={window}")
                    continue

if __name__ == "__main__":
    loop_all_models()


Training NBEATS on Window = 5
Loaded y scaler for window 5
[Hyperparameter Tuning] Running Optuna optimization for window=5
    [Hyperparameter Tuning] Running Optuna optimization for window=5
    [Device Setting] Using CPU for hyperparameter tuning (faster for small windows)
    [Tuning Completed] Switching back to mps for model training
    [Optuna] NBEATS best_params={'stacks': 2, 'blocks_per_stack': 1, 'batch_size': 64, 'learning_rate': 1.289795048085554e-05, 'max_epochs': 25, 'warm_start_epochs': 15, 'layers': 2, 'layer_size': 128}
[Optuna] NBEATS best_params={'stacks': 2, 'blocks_per_stack': 1, 'batch_size': 64, 'learning_rate': 1.289795048085554e-05, 'max_epochs': 25, 'warm_start_epochs': 15, 'layers': 2, 'layer_size': 128}
Applied inverse scaling: pred range [-0.016936, 0.015704], test range [-0.100574, 0.108696]
Completed NBEATS w=5: MSE=0.000268, Dir_Acc=0.5235
[Shared Hyperparams] Optimized on window=5: {'stacks': 2, 'blocks_per_stack': 1, 'batch_size': 64, 'learning_rate': 